# 장르 분포를 기준으로 Playlist를 추천해주는 모델 만들기
1. INPUT `playlist`, OUTPUT `playlist`      
2. 기준 : `장르분포도`  
3. 방식 : `ALS`    
4. 데이터 : `implicit`

In [1]:
# !pip install implicit

## 모듈 불러오기

In [2]:
# 모듈 불러오기
import os
import json
import numpy as np
import pandas as pd
import scipy.sparse as spr
from tqdm.notebook import tqdm

from itertools import chain
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import implicit

## 데이터 불러오기

In [3]:
# 데이터 불러오기
with open('data/train.json',encoding='utf-8-sig') as f:
    train_dict = json.load(f)
    
with open('data/song_meta.json',encoding='utf-8-sig') as f:
    song_dict = json.load(f)
    
with open('data/genre_gn_all.json',encoding='utf-8-sig') as f:
    genre_dict = json.load(f)
    
train = pd.DataFrame.from_dict(train_dict)
genre_gn_all = pd.read_json('data/genre_gn_all.json', typ = 'series') # 장르 읽어오기
song = pd.DataFrame.from_dict(song_dict)
genre = pd.DataFrame(genre_gn_all, columns = ['gnr_name']).reset_index().rename(columns = {'index' : 'gnr_code'}) # 장르코드 : gnr_code, 장르명 : gnr_name 


# 데이터 카피하기
df_train = train.copy()
df_song = song.copy()
# genre 데이터에서 사용할 컬럼만 가져오기
df_gnr_id_song = song[['song_gn_dtl_gnr_basket','id']].copy()

# 컬럼 알아보기 쉽게 한글로 바꾸기
df_gnr_id_song = df_gnr_id_song.rename(columns={'song_gn_dtl_gnr_basket':'장르','id':'곡id'})

---
dataframe 정리 :        
    `train` : 115072개의 플레이리스트    
    `song` : 707990개의 노래     
    `genre` : 255개의 세부 장르     
    `df_gnr_id_song` : 곡id와 장르id를 가진 데이터

# df_train에 `장르`, `장르id`, `장르count` 컬럼을 추가한 json 파일로 만들기
1. 저장한 파일명 : `df_train_with_gnr_cnt.json`     

In [4]:
from itertools import chain
genre_code_list = list(genre_dict.keys())
genre_id_dict = dict(zip(genre_code_list,range(len(genre_code_list)))) # {code : code_id}

# 노래 : 장르 딕셔너리 만들기 # {song_id : [gnr_id,gnr_id2...]}
songid_genre_dict = dict(zip(df_gnr_id_song['곡id'].tolist(),df_gnr_id_song['장르'].tolist()))

# df_train['장르']
song_id_lists = df_train['songs'].tolist() # playlist의 [[song_id, song_id2 ...],[ ...]]
song_genre_lists = [] 

for song_id_list in song_id_lists:  
    temp = [] 
    for song_id in song_id_list: 
        temp.append(songid_genre_dict[song_id]) 
    song_genre_lists.append(list(chain.from_iterable(temp))) # 이중리스트를 플랫하게 만듦.

# df_train['장르']는 중복을 포함하여 플레이리스트 수록곡의 모든 장르를 리스트로 묶은 컬럼
df_train['장르'] = song_genre_lists # playlist의 [[gnr, gnr2...],[ ... ]]

# df_train[장르id]의 id 
df_train['장르id'] = df_train['장르'].map(lambda x : [ genre_id_dict[i] for i in x]) 
# df_train.head(2)

# df_train['장르']의 개수를 중복과 관계없이 횟수를 센 결과.
df_train['장르cnt'] = df_train['장르'].apply(lambda x : len(x))
# df_train.head(2)

# 장르cnt가 0인 것들 임의의 값 주기 # 나중에 전체 나눗셈을 할 때 분모가 0일 경우 Null값이 생기기 때문
df_train['장르cnt'].replace(0,1, inplace=True)
# df_train[df_train['장르cnt'] == 0] # 잘 없어졌는지 확인.


# json파일로 만들기
# df_train.to_json('data/df_train_with_gnr_cnt.json') # 파일 생성
display(df_train)

,tags,id,plylst_title,songs,like_cnt,updt_date,장르,장르id,장르cnt
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,"[GN1402, GN1401, GN0901, GN0902, GN1001, GN101...","[111, 110, 57, 58, 66, 77, 70, 66, 78, 57, 58,...",48
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,"[GN0101, GN0103, GN0601, GN0605, GN0104, GN010...","[1, 3, 33, 37, 4, 1, 33, 37, 4, 1, 33, 37, 4, ...",105
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,"[GN0401, GN0403, GN0401, GN0403, GN0501, GN060...","[19, 21, 19, 21, 23, 33, 25, 38, 31, 19, 21, 1...",80
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,"[GN0908, GN1509, GN0901, GN2207, GN1501, GN150...","[64, 126, 57, 205, 118, 123, 1, 3, 63, 57, 101...",104
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,"[GN0101, GN0101, GN0103, GN2502, GN2506, GN250...","[1, 1, 3, 227, 231, 226, 9, 7, 9, 7, 228, 10, ...",145
...,...,...,...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000,"[GN1006, GN1013, GN1001, GN1007, GN1013, GN100...","[71, 78, 66, 72, 78, 66, 72, 78, 66, 72, 78, 6...",30
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000,"[GN1104, GN1101, GN1104, GN1102, GN1101, GN110...","[84, 81, 84, 82, 81, 84, 83, 81, 84, 83, 81, 8...",28
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000,"[GN0105, GN0101, GN0105, GN0101, GN2502, GN060...","[5, 1, 5, 1, 227, 33, 226, 37, 1, 19, 21, 4, 1...",25
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000,"[GN1107, GN1102, GN1101, GN1013, GN1008, GN100...","[87, 82, 81, 78, 73, 66, 66, 120, 118, 73, 68,...",106


---

# df_train DF sparse matrix 만들기 (npz 저장하기)
npz로 저장해서 모델에 적용할 테이블을 바로바로 가져오기 위함임.    


`spr_ply_gnr` : 115071(플레이리스트)x254(장르) sparse matrix

In [5]:
from collections import defaultdict, Counter

my_dict = defaultdict(dict)
genre_id_lists = df_train['장르id'].tolist() # genre_id_lists   # 이중 리스트 [[111,110,57 ....]]

# sparse matrix를 만들기 위한 row, col, dat 데이터 뽑아오기
for i in range(len(df_train)):
    my_dict[i] = dict(Counter(genre_id_lists[i]))
row =[] # playlist
col =[] # genre
dat =[] # gnr_cnt

for k, v in my_dict.items():
    for vk,vv in v.items():
        row.append(k)
        col.append(vk)
        dat.append(vv)

# print(row)
# print(col)
# print(dat)


spr_ply_gnr = spr.csr_matrix((dat, (row, col)), shape=(len(df_train), len(genre)))
spr_ply_gnr  # row = ply, col = genre, dat = gnr_cnt

# row별로 장르cnt를 나눠 각 곡의 개수와 상관없이 비율을 확인할 수 있도록 만듦
spr_ply_gnr_pct = spr_ply_gnr / df_train[['장르cnt']].values  # spr_ply_gnr_pct = matrix 형태


spr_spr_ply_gnr_pct= spr.csr_matrix(spr_ply_gnr_pct) # sparse matrix 형태로 저장

# npz 저장하기
# spr.save_npz('data/spr_spr_ply_gnr_pct.npz', spr_spr_ply_gnr_pct) 

`genre_id_lists` : 
`spr_ply_gnr` : 
`spr_ply_gnr_pct` :

---

# 저장된 npz 불러와서 플레이리스트간의  코사인 유사도 Sample 구하기


> 유사한 장르 분포를 가진 플레이리스트(user로 가정)간을 비교하기에 앞서서 `장르 cnt의 비율`로 만든 이유는       
장르 개수 등 코사인 유사도 계산에 영향을 줄 수 있는 `플레이리스트 내 장르`를 `플레이리스트 장르의 총합 개수`로 나누어 비율로 계산함.     
(ex) 비슷한 장르 분포를 가졌지만 A(Playlist)에 포함된 곡의 장르가 1000개, B(Playlist)에 포함된 곡의 장르가 10개라고 할 때)          
즉, 모든 Row의 데이터(장르 분포) 값을 더했을 때,총합이 1이 되도록 `비중(%)`으로 구함.    

In [6]:
# npz 불러오기 
spr_spr_ply_gnr_pct = spr.load_npz('data/spr_spr_ply_gnr_pct.npz') # csr형태

spr_ply_gnr_pct = spr_spr_ply_gnr_pct.todense() # matrix 형태

from sklearn.metrics.pairwise import cosine_similarity
# cosine_similarity(spr_ply_gnr_pct[1],spr_ply_gnr_pct[3]) # matrix 형태
cosine_similarity(spr_spr_ply_gnr_pct[1],spr_spr_ply_gnr_pct[4]) # csr 형태

array([[0.04929709]])

# als_model 피클 저장(따로 ipynb 만들어놓음 - 10.27일자)
`spr_spr_ply_gnr_pct.npz` 가져와서 만듦.      
implicit 모듈

`spr_spr_ply_gnr_pct` : 115071x254 sparse matrix     
`user_items` : 254x115071 sparse matrix

In [7]:
# als_model 추천서비스
import implicit

#데이터 불러오기
spr_spr_ply_gnr_pct = spr.load_npz('data/spr_spr_ply_gnr_pct.npz') # csr형태

als_model = implicit.als.AlternatingLeastSquares(factors=200, random_state=42) # factors = 200
als_model.fit(spr_spr_ply_gnr_pct*40) # 40을 곱해주는건 랜덤임
user_items = spr_spr_ply_gnr_pct.T.tocsr()


  0%|          | 0/15 [00:00<?, ?it/s]

In [8]:
# 
related = als_model.similar_items(44159, N=10)
related

[(44159, 1.0000001),
 (110897, 0.99502945),
 (38952, 0.99442804),
 (99281, 0.90095586),
 (48300, 0.89137363),
 (66830, 0.890818),
 (10663, 0.88890815),
 (28423, 0.88879424),
 (2667, 0.8760088),
 (41363, 0.8697529)]

In [9]:
# pickle로 저장하기
import pickle
with open('data/ply2ply_als_model.pickle','wb') as fw:
    pickle.dump(als_model, fw)

# model pickle(ply2ply_als_model) 불러오기 + top5 playlist 뽑아보기
- 11번 플레이리스트를 로그인한 회원 유저로 가정하고 이 회원유저와 유사한 장르 분포를 가진 다른 플레이리스트를 내림차순으로 정렬해 보여준다.
- 이 추천시스템은 유저의 음악 취향을 장르로 구분할 수 있다고 먼저 가정한다.    
    (Melon은 2019년 장르 개편을 통해 대장르 30개, 세부장르 191개로 나누어 곡의 세분 카테고리를 지정했으며 2020년 4월 분석에 사용된 카카오 아레나 데이터의 장르는 총 254개로 더욱 세분화하여 나타내고 있다.)           
- 그러나 유저의 플레이리스트 속에는 한 장르만 가지는 것이 아닐 뿐만아니라 한 곡이 다수의 대장르 또는 세부장르를 가지는 것을 확인할 수 있었다.    
- 이로 인해 한 가지 장르만으로 유저 집단을 나누는 것이 어려움이 있었다.    
- 이를 바탕으로 이 추천시스템에서는 같은 음악 장르를 좋아하면서 그 수록곡의 장르 분포가 비슷하다면 유저들의 취향을 더욱 구체적으로 나타낼 것이라는 가정으로 만들게 되었다.    
- 이 추천시스템의 장점은 기존 추천시스템과 다르게 유저의 특정 선호 가수, 노래 등의 유사도를 보이는 것이 아니라 장르 분포를 기준으로 협업필터링하기 때문에 더욱 다채로운 추천이 가능하다는 점이 있다.     

In [10]:
# model pickle(ply2ply_als_model) 불러오기
import pickle
with open('data/ply2ply_als_model.pickle','rb') as f:
    ply2ply_als_model = pickle.load(f)

In [11]:
# # genre확인
# i = int(genre[genre['gnr_name'] =='아이돌'].index.values)
# genre.iloc[i:i+5]

- 곡 중복 비율을 확인하는 이유는 유저가 이미 가지고 있는 곡의 비율을 확인하기 위함이고, 낮을수록 좋은 추천으로 평가한다.   
- 가수 중복 비율을 확인하는 이유는 유저가 이미 선호하는 가수의 비율을 확인하기 위함이고, 낮을수록 좋은 추천으로 평가한다.  
- 한계점 :      
    1) 회원 유저가 세분화된 특정 장르의 플레이리스트를 가지고 있는 경우     
        ex) `엑소`, `2018년 평창올림픽 노래`만 수록한 회원유저의 경우 다채로운 추천이 어렵다.
    2) 회원 유저의 플레이리스트에 많은 수록곡을 가지고 있는 경우     
        ex) 회원 유저의 수록곡의 크기가 추천된 플레이리스트의 수록곡보다 현저히 많은 경우, 중복곡 또는 중복 가수의 비율이 높게 나올 가능성이 높다.
        


In [13]:
# 11번 playlist의 top5 뽑아보기
als_model_5ply = als_model.similar_items(11,N=6)
print('input ply output ply',als_model_5ply)

# a는 als model결과로 나온 playlist index
a,b = zip(*als_model_5ply)

list_ply_id = list(a)
display(df_train.loc[list_ply_id])

#
x = pd.DataFrame([song.loc[x] for x in sorted(df_train.loc[11]['songs'])])
y = pd.DataFrame([song.loc[x] for x in sorted(df_train.loc[38216]['songs'])])

# x['artist_name_basket'].tolist()
import itertools
x_artist_lst= list(itertools.chain.from_iterable(x['artist_name_basket'].tolist()))
y_artist_lst= list(itertools.chain.from_iterable(y['artist_name_basket'].tolist()))

print('input playlist', Counter(x_artist_lst))
print('\n output playlist', Counter(y_artist_lst))

# 곡 중복 비율
print('\n곡 중복 비율 :',len(set(x.index) & set(y.index)) / len(set(x.index)))

# 가수 중복 비율
print('가수 중복 비율 :',len(set(x_artist_lst) & set(y_artist_lst)) / len(set(x_artist_lst)),'\n')

# print(x['artist_name_basket'].tolist())
# print('\n',y['artist_name_basket'].tolist())
# y[['song_name','artist_name_basket']].head()

input ply output ply [(11, 1.0), (46163, 0.79275084), (22985, 0.79132104), (23250, 0.79116994), (73248, 0.79039675), (28856, 0.78932935)]


,tags,id,plylst_title,songs,like_cnt,updt_date,장르,장르id,장르cnt
11,"[새해, 여행, 프로필음악, 카카오톡, 기분전환, 소원, 프로필, 소망, 다짐, 카톡]",151693,"노래로 의지를 불태우자! ""1일1다짐"" st용 프로필뮤직","[658738, 698779, 124485, 512610, 310028, 17422...",42,2017-02-16 15:45:23.000,"[GN0301, GN0805, GN0501, GN0502, GN0801, GN050...","[13, 55, 23, 24, 51, 31, 5, 1, 5, 1, 230, 228,...",138
46163,"[너를위해, 위로, 힐링, 플레이리스트, 힘들때]",69609,♪ 지친 너를 위한 노래 ♪,"[175073, 124485, 10203, 507808, 590379, 212310...",3,2017-06-13 09:54:25.000,"[GN0601, GN0606, GN0105, GN0101, GN0205, GN150...","[33, 38, 5, 1, 11, 118, 7, 121, 5, 1, 15, 13, ...",67
22985,[댄스],97040,good3,"[663602, 559041, 424712, 599327, 554859, 51780...",1,2013-09-02 22:10:27.000,"[GN0303, GN0301, GN0303, GN0301, GN0205, GN020...","[15, 13, 15, 13, 11, 7, 31, 33, 25, 38, 23, 1,...",106
23250,[댄스],139061,good3,"[82516, 9015, 589882, 492810, 290655, 49785, 2...",0,2013-08-21 10:16:23.000,"[GN0201, GN1501, GN0101, GN1504, GN2502, GN020...","[7, 118, 1, 121, 227, 11, 226, 231, 7, 11, 7, ...",106
73248,[댄스],18150,good3,"[87665, 454979, 12681, 82516, 9015, 589882, 49...",0,2013-09-19 00:55:33.000,"[GN2503, GN2501, GN2506, GN0201, GN0105, GN010...","[228, 226, 231, 7, 5, 1, 23, 16, 27, 13, 7, 11...",106
28856,[댄스],86153,good3,"[638927, 87665, 454979, 12681, 82516, 9015, 58...",0,2013-09-08 03:19:11.000,"[GN2503, GN0205, GN2501, GN2506, GN0201, GN250...","[228, 11, 226, 231, 7, 228, 226, 231, 7, 5, 1,...",106


input playlist Counter({'김동률': 2, '에일리': 2, '어쿠스틱 콜라보': 1, '별': 1, 'WINNER': 1, 'WABLE (와블)': 1, '서영은': 1, '미스에스': 1, '청년실업': 1, '처진 달팽이 (유재석 & 이적)': 1, '15& (박지민, 백예린)': 1, '안녕하신가영': 1, '이적': 1, '페퍼톤스 (Peppertones)': 1, '정인': 1, '윤종신': 1, '타코앤제이형': 1, '메이트': 1, 'f(x)': 1, 'SHINee (샤이니)': 1, '김건모': 1, '러브홀릭스': 1, 'MC 스나이퍼': 1, '장기하와 얼굴들': 1, '데프콘': 1, '이지형': 1, '제이레빗(J Rabbit)': 1, '에픽하이 (EPIK HIGH)': 1, '곽진언': 1, '이진아': 1, '크래쉬': 1, '핫펠트 (HA:TFELT)': 1, '장미여관': 1, '박효신': 1, '토이': 1, '옥상달빛': 1, '창모 (CHANGMO)': 1, 'DJ DOC': 1, '브로콜리너마저': 1, '임재범': 1, '한요한': 1, '커피소년': 1, '박보람': 1, '헬로봉주르': 1})

 output playlist Counter({'Crush': 4, 'pH-1': 2, 'Sofa': 1, '베이식 (Basick)': 1, 'SAAY': 1, 'DEAN': 1, 'GLABINGO (글라빙고)': 1, 'Hash Swan': 1, 'dkash (디캐시)': 1, 'WINNER': 1, '식케이 (Sik-K)': 1, '조승연': 1, '디아비 (D.I.B)': 1, '니화 (NiiHWA)': 1, 'J.Kill': 1, 'Timmy Room': 1, 'MINO (송민호)': 1, 'DPR LIVE': 1, 'Far East Movement': 1, 'OuiOui (위위)': 1, '그루비룸 (GroovyRoom)': 1, 'Evo': 1, '차콜 (Charcoal)': 1, 'Sleek 

---

# 추천시스템의 곡/가수 중복 비율을 확인한다.

x = `input playlist`, 로그인 한 회원유저로 가정한다.       
y1-y5 = `output playlist`, 회원유저가 추천받는 플레이리스트        
`x_y1_song_pct` : x와 y1의 곡 중복 비율       
`x_artist_lst` : 회원유저의 플레이리스트에 속한 가수의 리스트      
`y1_artist_lst` : 추천된 플레이리스트에 속한 가수의 리스트     

In [14]:
# top10
als_model_5ply = als_model.similar_items(15,N=6)
print('input ply output ply',als_model_5ply)

a,b = zip(*als_model_5ply)

list_ply_id = list(a)

# display(df_train.loc[list_ply_id])
# 11은 input 나머지는 output ply id
list_ply_id

ply_lists_df = []
for i in list_ply_id :
    ply_lists_df.append(pd.DataFrame([song.loc[x] for x in sorted(df_train.loc[i]['songs'])]))

x = ply_lists_df[0]
y1,y2,y3,y4,y5 = ply_lists_df[1],ply_lists_df[2],ply_lists_df[3],ply_lists_df[4],ply_lists_df[5]

x_y1_song_pct = len(set(x.index) & set(y1.index)) / len(set(x.index))
x_y2_song_pct = len(set(x.index) & set(y2.index)) / len(set(x.index))
x_y3_song_pct = len(set(x.index) & set(y3.index)) / len(set(x.index))
x_y4_song_pct = len(set(x.index) & set(y4.index)) / len(set(x.index))
x_y5_song_pct = len(set(x.index) & set(y5.index)) / len(set(x.index))
x_y_song_pct = [x_y1_song_pct,x_y2_song_pct,x_y3_song_pct,x_y4_song_pct,x_y5_song_pct]

print(np.average(x_y_song_pct),np.mean(x_y_song_pct), np.min(x_y_song_pct), np.max(x_y_song_pct))

# 곡 중복 비율
print('곡 중복 비율 :',len(set(x.index) & set(y1.index)) / len(set(x.index)))
print('곡 중복 비율 :',len(set(x.index) & set(y2.index)) / len(set(x.index)))
print('곡 중복 비율 :',len(set(x.index) & set(y3.index)) / len(set(x.index)))
print('곡 중복 비율 :',len(set(x.index) & set(y4.index)) / len(set(x.index)))
print('곡 중복 비율 :',len(set(x.index) & set(y5.index)) / len(set(x.index)))

import itertools
x_artist_lst= list(itertools.chain.from_iterable(x['artist_name_basket'].tolist()))
y1_artist_lst= list(itertools.chain.from_iterable(y1['artist_name_basket'].tolist()))
y2_artist_lst= list(itertools.chain.from_iterable(y2['artist_name_basket'].tolist()))
y3_artist_lst= list(itertools.chain.from_iterable(y3['artist_name_basket'].tolist()))
y4_artist_lst= list(itertools.chain.from_iterable(y4['artist_name_basket'].tolist()))
y5_artist_lst= list(itertools.chain.from_iterable(y5['artist_name_basket'].tolist()))


# 가수 중복 비율
print('가수 중복 비율 :',len(set(x_artist_lst) & set(y1_artist_lst)) / len(set(x_artist_lst)))
print('가수 중복 비율 :',len(set(x_artist_lst) & set(y2_artist_lst)) / len(set(x_artist_lst)))
print('가수 중복 비율 :',len(set(x_artist_lst) & set(y3_artist_lst)) / len(set(x_artist_lst)))
print('가수 중복 비율 :',len(set(x_artist_lst) & set(y4_artist_lst)) / len(set(x_artist_lst)))
print('가수 중복 비율 :',len(set(x_artist_lst) & set(y5_artist_lst)) / len(set(x_artist_lst)))


input ply output ply [(15, 0.99999994), (23399, 0.7501544), (7129, 0.72309875), (50306, 0.72207147), (40461, 0.72201544), (58032, 0.72184324)]
0.021276595744680847 0.021276595744680847 0.0 0.06382978723404255
곡 중복 비율 : 0.0
곡 중복 비율 : 0.0
곡 중복 비율 : 0.0
곡 중복 비율 : 0.0425531914893617
곡 중복 비율 : 0.06382978723404255
가수 중복 비율 : 0.09375
가수 중복 비율 : 0.0
가수 중복 비율 : 0.03125
가수 중복 비율 : 0.125
가수 중복 비율 : 0.09375


## 중복 곡의 아이디 확인

In [15]:
# y별 중복 곡은 뭐가 있나?
print(len(set(x.index)), set(x.index)) # input data
print('\n',set(x.index) & set(y1.index))
print('\n',set(x.index) & set(y2.index))
print('\n',set(x.index) & set(y3.index))
print('\n',set(x.index) & set(y4.index))
print('\n',set(x.index) & set(y5.index))

47 {146310, 61450, 258957, 335118, 562575, 570638, 655888, 261524, 420377, 253594, 352039, 473639, 168361, 313901, 214577, 688690, 104628, 177460, 574134, 88253, 373181, 565821, 502720, 678078, 354887, 106953, 473162, 392779, 258508, 394965, 457943, 663256, 563802, 670044, 259807, 356575, 174950, 362347, 703599, 291570, 18803, 119795, 334453, 367732, 657909, 129018, 44030}

 set()

 set()

 set()

 {129018, 174950}

 {129018, 258508, 88253}


## 중복곡 정보 확인 

In [18]:
song[(song['id']==88253) | (song['id']==174950)|(song['id']==258508)|(song['id']==420377)|(song['id']==563802)|(song['id']==655888)]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
88253,"[GN1003, GN1013, GN1001]",20030930,Absolution,32804,[100545],Stockholm Syndrome,[GN1000],[Muse],88253
174950,"[GN1503, GN1007, GN1501, GN1001]",20090623,트랜스포머: 패자의 역습 OST,593966,[105050],Burn It To The Ground,"[GN1500, GN1000]",[Nickelback],174950
258508,"[GN1013, GN1001]",20010701,Origin Of Symmetry,2720,[100545],Plug In Baby,[GN1000],[Muse],258508
420377,"[GN1013, GN1001]",20090908,Love Drunk,660551,[228896],Love Drunk,[GN1000],[Boys Like Girls],420377
563802,"[GN1013, GN1001]",20161202,"Rise And Fall, Rage And Grace",386978,[104372],"You`re Gonna Go Far, Kid",[GN1000],[The Offspring],563802
655888,[GN1201],20100621,Recovery,931351,[1986],Love The Way You Lie (Feat. Rihanna),[GN1200],[Eminem],655888


## 중복 가수 정보 확인

In [19]:
# 중복 가수는 누가 있나?
print(Counter(x_artist_lst)) # input data
print('\n',Counter(y1_artist_lst),'\n\n',Counter(y2_artist_lst),'\n\n',Counter(y3_artist_lst),'\n\n',Counter(y4_artist_lst),'\n\n',Counter(y5_artist_lst),'\n') #output data
print((Counter(x_artist_lst) & Counter(y1_artist_lst)).keys()) # (x,y1) 중복 가수
print((Counter(x_artist_lst) & Counter(y2_artist_lst)).keys()) # (x,y2) 중복 가수
print((Counter(x_artist_lst) & Counter(y3_artist_lst)).keys()) # (x,y3) 중복 가수
print((Counter(x_artist_lst) & Counter(y4_artist_lst)).keys()) # (x,y4) 중복 가수
print((Counter(x_artist_lst) & Counter(y5_artist_lst)).keys()) # (x,y5) 중복 가수



Counter({'Muse': 5, 'Bruno Mars': 5, 'Maroon 5': 4, 'Kelly Clarkson': 4, 'David Guetta': 2, 'Stryper': 2, 'The Offspring': 2, 'Ylvis': 1, 'Green Day': 1, 'Avril Lavigne': 1, 'Skid Row': 1, 'Nirvana': 1, 'Mr.Big': 1, 'Firehouse': 1, 'Nickelback': 1, 'Karina': 1, 'Justin Bieber': 1, 'Taio Cruz': 1, 'Pendulum': 1, 'Nari & Milani': 1, 'Cristian Marchi': 1, 'Bingo Players': 1, 'The Ting Tings': 1, 'Boys Like Girls': 1, 'Ne-Yo': 1, 'Korpiklaani': 1, 'Eminem': 1, 'Jeff Bernat': 1, 'Hey Monday': 1, 'Jay Sean': 1, 'DJ Gollum': 1, 'Empyre One': 1})

 Counter({'Lana Del Rey': 4, 'Meghan Trainor': 3, 'Kings Of Convenience': 3, 'Elle King': 3, 'HONNE': 3, 'My Chemical Romance': 2, 'Alessia Cara': 2, 'Imagine Dragons': 2, 'Nathan Sykes': 2, 'Green Day': 2, 'Future': 2, 'Icona Pop': 2, 'Calvin Harris': 1, 'Kungs': 1, 'Cookin` On 3 Burners': 1, '5 Seconds Of Summer': 1, 'John Legend': 1, 'Fitz & The Tantrums': 1, 'Bruno Mars': 1, 'Isac Elliot': 1, 'Elli Ingram': 1, 'Jason Mraz': 1, 'Frenship': 1, 'Cap

---

# 한 장르 안에 곡이 몇 개고, 가수가 몇 명인지
곡, 가수 유사도가 생각보다 낮음.    
현재 나눈 장르 기준 255
15명아티스트
한 장르에 10명의 아티스트
1/4

# Playlist 추천시스템  완성

In [24]:
# ply2ply_als_model pickle 불러오기
def ply2ply_recommendation(USER_ID): 
    import pickle
    with open('data/ply2ply_als_model.pickle','rb') as f:
        als_model = pickle.load(f)
    find_top5_ply = als_model.similar_items(USER_ID,N=6)
    top5 = list(map(lambda x : x[0], find_top5_ply))
    del top5[0]
    return top5

ply2ply_recommendation(10000)

[93827, 96340, 104385, 51419, 2909]